# Inference notebook

## Setup

### Imports

In [11]:
import os
import json
import warnings
from os.path import join
from itertools import pairwise, product

import torch
import numpy as np
import pandas as pd
import polars as pl
from numpy import ndarray
from torch import nn, Tensor
import plotly.express as px
from scipy.spatial.transform import Rotation
from pandas import DataFrame as DF
from kagglehub import competition_download, dataset_download, model_download

import kaggle_evaluation.cmi_inference_server

### Supress performance warngings

In [12]:
warnings.filterwarnings(
    "ignore",
    message=(
        "DataFrame is highly fragmented.  This is usually the result of "
        "calling `frame.insert` many times.*"
    ),
    category=pd.errors.PerformanceWarning,
)

### Config

In [13]:
GRAVITY_WORLD = np.array([0, 0, 9.81])
ACCELRATION_COLS = ["acc_x", "acc_y", "acc_z"]
META_DATA_COLUMNS = [
    'row_id',
    'sequence_type',
    'sequence_id',
    'sequence_counter',
    'subject',
    'orientation',
    'behavior',
    'phase',
    'gesture',
]
EULER_ANGLES_COLS = ["euler_x", "euler_y", "euler_z"]
QUATERNION_COLS = ['rot_w', 'rot_x', 'rot_y', 'rot_z']
GRAVITY_FREE_ACC_COLS = ["gravity_free_" + col for col in ACCELRATION_COLS]

## Preprocessing

### Load dataset meta data

In [14]:
meta_data_path = dataset_download(
    handle="mauroabidalcarrer/prepocessed-cmi-2025",
    path="preprocessed_dataset/full_dataset_meta_data.json"
)
with open(meta_data_path, "r") as fp:
    meta_data = json.load(fp)

### Define function to get the feature columns
Feature columns change over time so it's better to have a function to get them than manually update a variable every time we add/remove features.

In [15]:
def get_feature_cols(df:DF) -> list[str]:
    return list(set(df.columns) - set(META_DATA_COLUMNS) - set(meta_data["target_names"]))

### Define preprocessing function

In [16]:

def get_fillna_val_per_feature_col(df:DF) -> dict:
    return {col: 1.0 if col == 'rot_w' else 0 for col in get_feature_cols(df)}

def imputed_features(df:DF) -> DF:
    # Missing ToF values are already imputed by -1 which is inconvinient since we want all missing values to be NaN.    
    # So we replace them by NaN and then perform imputing.  
    tof_vals_to_nan = {col: -1.0 for col in df.columns if col.startswith("tof")}
    # fillna_val_per_col = {col: 1.0 if col == 'rot_w' else 0 for col in df.columns}
    #print(get_fillna_val_per_feature_col(df))
    df[get_feature_cols(df)] = (
        df
        .loc[:, get_feature_cols(df)]
        # df.replace with np.nan sets dtype to floar64 so we set it back to float32
        .replace(tof_vals_to_nan, value=np.nan)
        .astype("float32")
        .groupby(df["sequence_id"], observed=True, as_index=False)
        .ffill()
        .groupby(df["sequence_id"], observed=True, as_index=False)
        .bfill()
        # In case there are only nan in the column in the sequence
        .fillna(get_fillna_val_per_feature_col(df))
    )
    return df

def rot_euler_angles(df:DF) -> ndarray:
    df[EULER_ANGLES_COLS] = (
        Rotation
        .from_quat(df[QUATERNION_COLS])
        .as_euler("xyz")
        .squeeze()
    )
    return df


def add_gravity_free_acc_cols(df:DF) -> DF:
    # Vectorized version of https://www.kaggle.com/code/wasupandceacar/lb-0-82-5fold-single-bert-model#Dataset `remove_gravity_from_acc`
    rotations = Rotation.from_quat(df[QUATERNION_COLS])
    gravity_sensor_frame = rotations.apply(GRAVITY_WORLD, inverse=True)
    df[GRAVITY_FREE_ACC_COLS] = df[ACCELRATION_COLS] - gravity_sensor_frame
    return df

def agg_tof_cols_per_sensor(df:DF) -> DF:
    for tof_idx in range(1, 6):
        tof_name = f"tof_{tof_idx}"
        tof_cols = [f"{tof_name}_v{v_idx}" for v_idx in range(64)]
        if any(map(lambda col: col not in df.columns, tof_cols)):
            print(f"Some (or) all ToF {tof_idx} columns are not in the df. Maybe you already ran this cell?")
            continue
        df = (
            df
            # Need to use a dict otherwise the name of the col will be "tof_preffix" instead of the value it contains
            .assign(**{tof_name:df[tof_cols].mean(axis="columns")})
            .drop(columns=tof_cols)
        )
    return df

def add_diff_features(df:DF) -> DF:
    df[[col + "_diff" for col in get_feature_cols(df)]] = (
        df
        .groupby("sequence_id", observed=True)
        [get_feature_cols(df)]
        .diff()
        .fillna(get_fillna_val_per_feature_col(df))
        .values
    )
    return df

def length_normed_sequence_feat_arr(sequence: DF) -> ndarray:
    features = (
        sequence
        .loc[:, meta_data["get_feature_cols(df)"]]
        .values
    )
    normed_sequence_len = meta_data["pad_seq_len"]
    len_diff = abs(normed_sequence_len - len(features))
    if len(features) < normed_sequence_len:
        padded_features = np.pad(
            features,
            ((len_diff // 2 + len_diff % 2, len_diff // 2), (0, 0)),
        )
        return padded_features
    elif len(features) > normed_sequence_len:
        return features[len_diff // 2:-len_diff // 2]
    else:
        return features

def preprocess_sequence(sequence_df:pl.DataFrame) -> ndarray:
    return (
        sequence_df                     
        .to_pandas()                            # Convert to pandas dataframe.
        .pipe(imputed_features)                 # Impute missing data.
        .pipe(rot_euler_angles)                 # Add rotation acc expressed as euler angles.
        .pipe(add_gravity_free_acc_cols)        # Add gravity agnostic acceleration.
        .pipe(agg_tof_cols_per_sensor)          # Aggregate ToF columns.
        .pipe(add_diff_features)                # 
        .loc[:, meta_data["get_feature_cols(df)"]]      # Retain only the usefull columns a.k.a features.
        .sub(meta_data["mean"])                 # Subtract features by their mean, std norm pt.1.
        .div(meta_data["std"])                  # Divide by Standard deviation, std norm pt.2.
        .pipe(length_normed_sequence_feat_arr)  # get feature ndarray of sequence.
        .T                                      # Transpose to swap channel and X dimensions.
    )

## Load model

### Define model

In [17]:
class ResidualBlock(nn.Module):
    def __init__(self, in_chns:int, out_chns:int):
        super().__init__()
        self.blocks = nn.Sequential(
            nn.Conv1d(in_chns, out_chns, kernel_size=3, padding=1),
            nn.BatchNorm1d(out_chns),
            nn.ReLU(),
            nn.Conv1d(out_chns, out_chns, kernel_size=3, padding=1),
            nn.BatchNorm1d(out_chns),
        )
        if in_chns == out_chns:
            self.skip_connection = nn.Identity() 
        else:
            # TODO: set bias to False ?
            self.skip_connection = nn.Sequential(
                nn.Conv1d(in_chns, out_chns, 1),
                nn.BatchNorm1d(out_chns)
            )

    def forward(self, x:Tensor) -> Tensor:
        activaition_maps = self.skip_connection(x) + self.blocks(x)
        return nn.functional.relu(activaition_maps)

class Resnet(nn.Module):
    def __init__(
            self,
            in_channels:int,
            depth:int,
            # n_res_block_per_depth:int,
            mlp_width:int,
            n_class:int,
        ):
        super().__init__()
        chs_per_depth = [in_channels * 2 ** i for i in range(depth)]
        blocks_chns_it = pairwise(chs_per_depth)
        self.res_blocks = [ResidualBlock(in_chns, out_chns) for in_chns, out_chns in blocks_chns_it]
        self.res_blocks = nn.ModuleList(self.res_blocks)
        self.mlp_head = nn.Sequential(
            nn.LazyLinear(mlp_width),
            nn.ReLU(),
            nn.Linear(mlp_width, n_class),
            nn.Softmax(dim=1),
        )
        
        
    def forward(self, x:Tensor) -> Tensor:
        activation_maps = x
        for res_block in self.res_blocks:
            activation_maps = nn.functional.max_pool1d(res_block(activation_maps), 2)
        out = activation_maps.view(activation_maps.shape[0], -1)
        out = self.mlp_head(out)
        return out

model = Resnet(
    in_channels=46,
    depth=4,
    mlp_width=256,
    n_class=18,
)

### Load model weights

In [18]:
model_state_parent_dir = model_download("mauroabidalcarrer/cmi-resnet/pyTorch/trained_on_diff_features_cpu")
model_state_filename = os.listdir(model_state_parent_dir)[0]
model_state_path = join(model_state_parent_dir, model_state_filename)
model_weights_state_dict = torch.load(model_state_path, weights_only=True) #["model"]
model.load_state_dict(model_weights_state_dict)

<All keys matched successfully>

## Perform inference

In [19]:
def predict(sequence_df:pl.DataFrame, _:pl.DataFrame) -> str:
    x = preprocess_sequence(sequence_df)
    print(x.shape)
    x = torch.unsqueeze(Tensor(x), dim=0)
    y_pred = (
        model(x)
        .max(dim=1)[1]
        .numpy()
        .squeeze()
    )
    y_pred_str = meta_data["target_names"][y_pred]

    return y_pred_str

inference_server = kaggle_evaluation.cmi_inference_server.CMIInferenceServer(predict)
if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    competition_dataset_path = competition_download("cmi-detect-behavior-with-sensor-data")
    test_df = pl.read_csv(join(competition_dataset_path, "test.csv"))
    for seq_id, seq in test_df.group_by("sequence_id"):
        predict(seq, None)
    inference_server.run_local_gateway(
        data_paths=(
            join(competition_dataset_path, "test.csv"),
            join(competition_dataset_path, "test_demographics.csv"),
        )
    )
    inference_server = kaggle_evaluation.cmi_inference_server.CMIInferenceServer(predict)
    inference_server.run_local_gateway(
        data_paths=(
            join(competition_dataset_path, "train.csv"),
            join(competition_dataset_path, "train_demographics.csv"),
        )
    )

# inference_server.serve()

(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)
(46, 127)


KeyboardInterrupt: 